<h1> Basic encoders, bigram model </h1>

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
with open('input.txt', 'r') as f:
    text = f.read()

In [3]:
len(text)

1115394

In [4]:
text[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
vocab_size


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


65

Here, we just need some sort of encoder from character to integer. We could call gpt2 embeddings (vocab size ~50k) or we can just write our own small, simple tokenizer that just works off of a dictionary enumeration.

In [32]:
#for every char, int pair, define a dictionary of int to char and char to int
char_to_int = {char:i for i, char in enumerate(chars)}
int_to_char = {i:char for i, char in enumerate(chars)}

encode = lambda string : [char_to_int[char] for char in string]
decode = lambda integers : ''.join([int_to_char[i] for i in integers]) #''.join() joins together characters with nothing between them

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


Let's do some Pytorch now

In [8]:
#use torch.tensor to use pytorch
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
data[:1000]

torch.Size([1115394]) torch.int64


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  1, 41, 

In [9]:
#create train, val split

train_val_split = .9 #take 90% val

train_data = data[:int(len(data) * train_val_split)]
val_data = data[int(len(data) * train_val_split):]


To follow the video, we use block sizes/contexts of 8. Each block actually contains 8 different context examples

In [10]:
block_size = 8 #this is a maximimum num of context for a target
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1:]
for idx in range(block_size):
    context = x[:idx + 1]
    target = y[idx]
    print(f"when input is {context} target is {target}")

when input is tensor([18]) target is 47
when input is tensor([18, 47]) target is 56
when input is tensor([18, 47, 56]) target is 57
when input is tensor([18, 47, 56, 57]) target is 58
when input is tensor([18, 47, 56, 57, 58]) target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) target is 58


The following section will use grain for batch loading when working with the Zapbench dataset. For now, we just define each target batch as being +1 off of each context batch.

In [38]:
torch.manual_seed(1337)
batch_size = 4 #how many of the blocks are you looking at simultaneously

def get_batch(split):
    data = train_data if split == 'train' else val_data
    batch_start_index = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in batch_start_index])
    y = torch.stack([data[i+1: i + block_size+1] for i in batch_start_index])
    return x, y
xb, yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)

print('targets')
print(yb.shape)
print(yb)

print('-------')


inputs
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-------


An example of a language model, Bigram language model (think 2-grams)

In [37]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) #todo understand

    def forward(self, index, targets = None):
        logits = self.token_embedding_table(index) #todo understand

        if targets is not None:
            B,T,C = logits.shape #todo understand
            logits = logits.view(B*T, C) #todo understand
            targets= targets.view(B*T) #todo understand
            loss= F.cross_entropy(logits, targets)
        else:
            loss = None
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index) #todo understand
            logits = logits[:, -1, :] #todo understand
            probs = F.softmax(logits, dim=-1) #todo understand
            index_next = torch.multinomial(probs, num_samples=1) #todo understand
            index = torch.cat([index, index_next], dim=1) #todo understand
        return index
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)


index = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(index , max_new_tokens = 100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [40]:
optimizer = torch.optim.Adam(m.parameters(), lr= 1e-3)



In [109]:
batch_size = 32

for steps in range(1000000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.478142023086548


In [111]:
print(decode(m.generate(index , max_new_tokens = 500)[0].tolist()))




Stceat d he mis;
The, canern hisicand bed
S:
IESSILYomysm s: cour?
Ha Her I'Murpursuro lan'dr or her ily alld bit t.
tis h
' w hon at VOf rostth, llin hare for Whomstonon t lendin cewof cen.
O:
Ash grd ld.
S:
S: the w's omy anjend oy t ha rthenowheeato tillou Theere bo l! bomee he, perele rd teod ar br
HA sin kiathesor K:
S:
MEEd,
Of here?


CK: n rrthr Me car:
Bu,
GERDUCOF tho, thedom, her,
Myeme hoyon-dit INom hthathall hamere:
OPEShed,
Prut ata ddositeter;
Plilaisthte f alas:
Th ' wr fo ir;



<h1> Math trick behind self-attention </h1>


In [130]:
torch.manual_seed(1337)
B,T,C = 4, 8, 2  #batch, time, channel
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [143]:
xbag_of_words = torch.zeros((B,T, C))

for b in range(B):
    for t in range(T):
        x_prev = x[b, :t + 1]
        xbag_of_words[b, t] = torch.mean(x_prev, 0)

In [133]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [137]:
xbag_of_words[0] #each row is an avg of each previous row, so the firt element is the same

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [156]:
#quicker version using the below math trick

weight = torch.tril(torch.ones(T, T))
weight = weight/weight.sum(1, keepdim=True)
xbag_of_words2 = weight @ x # (T,T) @ B,T,C, add a B because of the @ batch multiplication symbol  --> B,T,C
torch.allclose(xbag_of_words, xbag_of_words2)

True

In [157]:
#even better softmax

triangle = torch.tril(torch.ones(T, T))
weight = torch.zeros((T,T))
weight = weight.masked_fill(triangle ==0, float('-inf'))
weight = F.softmax(weight, dim=-1)
xbag_of_words3 = weight @ x
torch.allclose(xbag_of_words, xbag_of_words3)

True

In [139]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [140]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
b = torch.randint(0,10,(3,2)).float() #pick nums between 0 and 10 for a 3x2 matrix
c = a @ b #multiply the  matrices
print('a =')
print(a)
print('b =')
print(b)
print('c =')
print(c)


a =
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
b =
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c =
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


Since we took out a number of the 1s, when we multiply it together, we can then divide by dimension and this allows us to take a average. First row is average of first row (itself), second is average of first two, third average of first 3, etc.

In [141]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a=  a/torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float() #pick nums between 0 and 10 for a 3x2 matrix
c = a @ b #multiply the  matrices
print('a =')
print(a)
print('b =')
print(b)
print('c =')
print(c)


a =
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b =
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c =
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
